In [32]:
import numpy as np
import pandas as pd

import math
from math import pi

import scipy.io
from scipy.interpolate import interp1d

## Constants

Needed for:
- temperature correction

In [33]:
T_s = 60.0 # Faranheit, standard reference temperature
C = 6.45e-6 # ft/ft/Faranheit, coefficient of expansion for mild steel

E = 2.9e+7 # psi, modulus of elasticity of steel
W = 62.3 # lbs/ft^3

m2ft_coeff = 3.28084 # meter to feet conversion coefficient `ft = m * m2ft_coeff`
um2in_coeff = 3.93700787e-5 # micrometer to inch conversion coefficient `in = um * um2in_coeff`

## Input data

### Temperatures

In [34]:
T_c = 68.0 # Faranheit, calibration temperature of Master Tape
T_st = 64.4 # Faranheit, strapping temperature

### Rings

In [35]:
rings_array = [
    {"idx": 0, "height": 8.26772, "thickness": 1.58, "strappingHeight": 0.1588,  "circumference": 502.6545},
    {"idx": 1, "height": 8.26772, "thickness": 1.58},
    {"idx": 2, "height": 8.26772, "thickness": 1.58},
    {"idx": 3, "height": 8.26772, "thickness": 1.58},
    {"idx": 4, "height": 8.26772, "thickness": 1.58},
]

rings_df = pd.DataFrame.from_records(rings_array, index="idx")

ring_count = len(rings_df)

rings_df

,height,thickness,strappingHeight,circumference
idx,,,,
0,8.26772,1.58,0.1588,502.6545
1,8.26772,1.58,NaN,NaN
2,8.26772,1.58,NaN,NaN
3,8.26772,1.58,NaN,NaN
4,8.26772,1.58,NaN,NaN


### Deadwood

#### Structural

In [36]:
structural_deadwood_add = [
    {"idx": 0, "from": "Below", "to": 0, "value": 2.0007},
    {"idx": 1, "from": 0, "to": 53.22, "value": 0.0486},
    {"idx": 2, "from": 41.14, "to": 106.6, "value": 0.0154},
]

structural_deadwood_deduct = [
    {"idx": 0, "from": 17.32, "to": 26.97, "value": 0.0005},
    {"idx": 1, "from": 1.23, "to": 301.23, "value": 0.0015},
]

#### Bottom

In [37]:
bottom_deadwood_add = [
    {"idx": 0, "from": "Below", "to": 0, "value": 411.5441},
]

bottom_deadwood_deduct = [
    {"idx": 0, "from": 0, "to": 1, "value": 17.1068},
    {"idx": 1, "from": 1, "to": 2, "value": 6.4007},
]

### Other inputs

In [38]:
G_obs = 1.0 # observed specific gravity of liquid in tank
G_s = 0.7389 # specific gravity at 60 F of the liquid to be stored in the tank
liquid_height = 23.1 # feet, liquid height in tank

### Robot readouts

`allStations.mat` contains two matrices:
- `height` shape(245, 1) - contains a list of heights at which the measurements took place
- `offset` shape(245, 8) - contains a list of offset measurements for each height (seems like millimeters, final design should use micrometers)

Since the values in the file are all in metric, they will be recalculated to imperial.

In [39]:
raw_data = scipy.io.loadmat('allStations.mat')

In [40]:
heights = raw_data["height"][:, 0] * m2ft_coeff
offsets = raw_data["offset"] * (1000 * um2in_coeff)

station_count = offsets.shape[1]

## Calculations

### Ring circumference calculation

Determining ring circumferences, based on the measured data.

In [41]:
ref_circumference = rings_df.loc[0]["circumference"]
ref_radius = (ref_circumference * 12) / (2 * pi) # inches, r = l / 2*pi

strap_height = rings_df.loc[0]["strappingHeight"] * rings_df.loc[0]["height"]

First, offsets need to be calibrated to the initial manual strapping measurement, so we assume, that at the place of strapping, the offeset is ~0. Then we create offset interpolators for every station, based on height.

In [42]:
offset_interpolators = []
for station_idx in range(station_count):
    initial_interpolator = interp1d(heights, offsets[:, station_idx])

    strap_height_offset = float(initial_interpolator(strap_height))
    
    offset_interpolators.append(interp1d(heights, offsets[:, station_idx] - strap_height_offset))

Next, we get the average radius for each ring, at 20% and at 80% of ring height.

In [43]:
traversed_height = rings_df.loc[0]["height"]

for ring_idx in range(1, ring_count):
    current_ring_height = rings_df.loc[ring_idx]["height"]
    h_20_percent = traversed_height + (current_ring_height * 0.2)
    h_80_percent = traversed_height + (current_ring_height * 0.8)

    station_radi_h20 = []
    station_radi_h80 = []
    for station_idx in range(len(offset_interpolators)):
        h20_offset = offset_interpolators[station_idx](h_20_percent)
        h80_offset = offset_interpolators[station_idx](h_80_percent)
        
        station_radi_h20.append(ref_radius + h20_offset)
        station_radi_h80.append(ref_radius + h80_offset)
    
    ring_h_20_mean_radius = np.array(station_radi_h20).mean()
    ring_h_80_mean_radius = np.array(station_radi_h80).mean()

    ring_mean_radius = (ring_h_20_mean_radius + ring_h_80_mean_radius) / 2

    rings_df.loc[ring_idx]["circumference"] = (2 * pi * ring_mean_radius )/ 12
    
    traversed_height = traversed_height + current_ring_height

rings_df["height_inch"] = rings_df["height"] * 12
rings_df["height_inch_total"] = rings_df["height_inch"]
rings_df["height_at_strap"] = rings_df["height_inch"]

for ring_idx in range(ring_count):
    if ring_idx == 0:
        rings_df.loc[ring_idx]["height_at_strap"] = rings_df.loc[ring_idx]["height"] * rings_df.loc[ring_idx]["strappingHeight"]
    else:
        rings_df.loc[ring_idx]["height_inch_total"] = rings_df.loc[ring_idx - 1]["height_inch_total"] + rings_df.loc[ring_idx]["height_inch"]
        rings_df.loc[ring_idx]["height_at_strap"] = (rings_df.loc[ring_idx - 1]["height_inch_total"] / 12) + (rings_df.loc[ring_idx]["height"] * 0.5)

rings_df

,height,thickness,strappingHeight,circumference,height_inch,height_inch_total,height_at_strap
idx,,,,,,,
0,8.26772,1.58,0.1588,502.654500,99.21264,99.21264,1.312914
1,8.26772,1.58,NaN,502.494401,99.21264,198.42528,12.401580
2,8.26772,1.58,NaN,502.528715,99.21264,297.63792,20.669300
3,8.26772,1.58,NaN,502.511144,99.21264,396.85056,28.937020
4,8.26772,1.58,NaN,502.586328,99.21264,496.06320,37.204740


### `B.1.2` Circumference temperature correction

In [44]:
correction_factor = (1 + ((T_s - T_st) * C)) * (1 + ((T_st - T_c) * C))

rings_df["temp_correction"] = rings_df["circumference"] - (rings_df["circumference"] * correction_factor)
rings_df["temp_corr_circumference"] = rings_df["circumference"] + rings_df["temp_correction"]
rings_df


,height,thickness,strappingHeight,circumference,height_inch,height_inch_total,height_at_strap,temp_correction,temp_corr_circumference
idx,,,,,,,,,
0,8.26772,1.58,0.1588,502.654500,99.21264,99.21264,1.312914,0.025937,502.680437
1,8.26772,1.58,NaN,502.494401,99.21264,198.42528,12.401580,0.025928,502.520329
2,8.26772,1.58,NaN,502.528715,99.21264,297.63792,20.669300,0.025930,502.554645
3,8.26772,1.58,NaN,502.511144,99.21264,396.85056,28.937020,0.025929,502.537073
4,8.26772,1.58,NaN,502.586328,99.21264,496.06320,37.204740,0.025933,502.612261


### `B.1.4` Empty tank basis correction

In [45]:
K = W / (24 * pi * E)

rings_df["empty_tank_correction"] = 0

for ring_idx in range(ring_count):
    h = liquid_height - rings_df.loc[ring_idx]["height_at_strap"]

    if h > 0: 
        c = rings_df.loc[ring_idx]["temp_corr_circumference"]
        t = rings_df.loc[ring_idx]["thickness"]

        rings_df.loc[ring_idx, "empty_tank_correction"] = ((-1) * K) * ((G_obs * h * math.pow(c, 2)) / t)
        
rings_df["empty_tank_circumference"] = rings_df["temp_corr_circumference"] + rings_df["empty_tank_correction"]

rings_df

,height,thickness,strappingHeight,circumference,height_inch,height_inch_total,height_at_strap,temp_correction,temp_corr_circumference,empty_tank_correction,empty_tank_circumference
idx,,,,,,,,,,,
0,8.26772,1.58,0.1588,502.654500,99.21264,99.21264,1.312914,0.025937,502.680437,-0.099278,502.581158
1,8.26772,1.58,NaN,502.494401,99.21264,198.42528,12.401580,0.025928,502.520329,-0.048719,502.471610
2,8.26772,1.58,NaN,502.528715,99.21264,297.63792,20.669300,0.025930,502.554645,-0.011071,502.543574
3,8.26772,1.58,NaN,502.511144,99.21264,396.85056,28.937020,0.025929,502.537073,0.000000,502.537073
4,8.26772,1.58,NaN,502.586328,99.21264,496.06320,37.204740,0.025933,502.612261,0.000000,502.612261


### `B.1.5` Internal circumference correction

In [46]:
rings_df["plate_thickness_deduction"] = (rings_df["thickness"] * pi)/6
rings_df["internal_circumference"] = rings_df["empty_tank_circumference"] - rings_df["plate_thickness_deduction"]

rings_df

,height,thickness,strappingHeight,circumference,height_inch,height_inch_total,height_at_strap,temp_correction,temp_corr_circumference,empty_tank_correction,empty_tank_circumference,plate_thickness_deduction,internal_circumference
idx,,,,,,,,,,,,,
0,8.26772,1.58,0.1588,502.654500,99.21264,99.21264,1.312914,0.025937,502.680437,-0.099278,502.581158,0.827286,501.753872
1,8.26772,1.58,NaN,502.494401,99.21264,198.42528,12.401580,0.025928,502.520329,-0.048719,502.471610,0.827286,501.644324
2,8.26772,1.58,NaN,502.528715,99.21264,297.63792,20.669300,0.025930,502.554645,-0.011071,502.543574,0.827286,501.716288
3,8.26772,1.58,NaN,502.511144,99.21264,396.85056,28.937020,0.025929,502.537073,0.000000,502.537073,0.827286,501.709787
4,8.26772,1.58,NaN,502.586328,99.21264,496.06320,37.204740,0.025933,502.612261,0.000000,502.612261,0.827286,501.784975


### `B.1.6` Addition of liquid head stress to internal circumferences

In [47]:
K = W / (24 * pi * E)

rings_df["stressed_correction"] = 0

for ring_idx in range(ring_count):
    h = 0

    if ring_idx == 0:
        h = (1 - rings_df.loc[ring_idx]["strappingHeight"]) * rings_df.loc[ring_idx]["height"]
    else:
        h = 0.5 * rings_df.loc[ring_idx]["height"]

    if h > 0: 
        c = rings_df.loc[ring_idx]["internal_circumference"]
        t = rings_df.loc[ring_idx]["thickness"]

        rings_df.loc[ring_idx, "stressed_correction"] = K * ((G_s * h * math.pow(c, 2)) / t)
        
rings_df["stressed_internal_circumference"] = rings_df["internal_circumference"] + rings_df["stressed_correction"]

rings_df

,height,thickness,strappingHeight,circumference,height_inch,height_inch_total,height_at_strap,temp_correction,temp_corr_circumference,empty_tank_correction,empty_tank_circumference,plate_thickness_deduction,internal_circumference,stressed_correction,stressed_internal_circumference
idx,,,,,,,,,,,,,,,
0,8.26772,1.58,0.1588,502.654500,99.21264,99.21264,1.312914,0.025937,502.680437,-0.099278,502.581158,0.827286,501.753872,0.023330,501.777203
1,8.26772,1.58,NaN,502.494401,99.21264,198.42528,12.401580,0.025928,502.520329,-0.048719,502.471610,0.827286,501.644324,0.013861,501.658185
2,8.26772,1.58,NaN,502.528715,99.21264,297.63792,20.669300,0.025930,502.554645,-0.011071,502.543574,0.827286,501.716288,0.013865,501.730154
3,8.26772,1.58,NaN,502.511144,99.21264,396.85056,28.937020,0.025929,502.537073,0.000000,502.537073,0.827286,501.709787,0.013865,501.723652
4,8.26772,1.58,NaN,502.586328,99.21264,496.06320,37.204740,0.025933,502.612261,0.000000,502.612261,0.827286,501.784975,0.013869,501.798844


### `B.1.7` Calculating incremental volume (BBL/inch)

In [48]:
rings_df["internal_radius"] = (rings_df["stressed_internal_circumference"]/(2 * pi)) * 12
rings_df["incremental"] = (pi * rings_df["internal_radius"].pow(2)) / 9702
rings_df

,height,thickness,strappingHeight,circumference,height_inch,height_inch_total,height_at_strap,temp_correction,temp_corr_circumference,empty_tank_correction,empty_tank_circumference,plate_thickness_deduction,internal_circumference,stressed_correction,stressed_internal_circumference,internal_radius,incremental
idx,,,,,,,,,,,,,,,,,
0,8.26772,1.58,0.1588,502.654500,99.21264,99.21264,1.312914,0.025937,502.680437,-0.099278,502.581158,0.827286,501.753872,0.023330,501.777203,958.323866,297.380995
1,8.26772,1.58,NaN,502.494401,99.21264,198.42528,12.401580,0.025928,502.520329,-0.048719,502.471610,0.827286,501.644324,0.013861,501.658185,958.096559,297.239939
2,8.26772,1.58,NaN,502.528715,99.21264,297.63792,20.669300,0.025930,502.554645,-0.011071,502.543574,0.827286,501.716288,0.013865,501.730154,958.234009,297.325230
3,8.26772,1.58,NaN,502.511144,99.21264,396.85056,28.937020,0.025929,502.537073,0.000000,502.537073,0.827286,501.709787,0.013865,501.723652,958.221591,297.317524
4,8.26772,1.58,NaN,502.586328,99.21264,496.06320,37.204740,0.025933,502.612261,0.000000,502.612261,0.827286,501.784975,0.013869,501.798844,958.365197,297.406647


### `B.1.8` Volumetric increase for each ring for each inch of liquid head above the ring

In [49]:
d = ((rings_df["internal_radius"] / 12) * 2).mean()

K = (pi * W * G_s * math.pow(d, 3)) / (4 * E)

rings_df["volumetric_increase"] = 0

for ring_idx in range(ring_count - 1):
    h = rings_df.loc[ring_idx]["height_inch"]
    t = rings_df.loc[ring_idx]["thickness"]

    rings_df.loc[ring_idx, "volumetric_increase"] = ((h / t) * K) / 9702

rings_df["volumetric_correction"] = 0

for ring_idx in range(1, ring_count):
    h = rings_df.loc[ring_idx]["height_inch"]
    t = rings_df.loc[ring_idx]["thickness"]

    rings_df.loc[ring_idx, "volumetric_correction"] = rings_df.loc[ring_idx - 1, "volumetric_correction"] + rings_df.loc[ring_idx - 1, "volumetric_increase"]

rings_df["corrected_increments"] = rings_df["incremental"] + rings_df["volumetric_correction"]

rings_df

,height,thickness,strappingHeight,circumference,height_inch,height_inch_total,height_at_strap,temp_correction,temp_corr_circumference,empty_tank_correction,empty_tank_circumference,plate_thickness_deduction,internal_circumference,stressed_correction,stressed_internal_circumference,internal_radius,incremental,volumetric_increase,volumetric_correction,corrected_increments
idx,,,,,,,,,,,,,,,,,,,,
0,8.26772,1.58,0.1588,502.654500,99.21264,99.21264,1.312914,0.025937,502.680437,-0.099278,502.581158,0.827286,501.753872,0.023330,501.777203,958.323866,297.380995,0.03287,0.000000,297.380995
1,8.26772,1.58,NaN,502.494401,99.21264,198.42528,12.401580,0.025928,502.520329,-0.048719,502.471610,0.827286,501.644324,0.013861,501.658185,958.096559,297.239939,0.03287,0.032870,297.272809
2,8.26772,1.58,NaN,502.528715,99.21264,297.63792,20.669300,0.025930,502.554645,-0.011071,502.543574,0.827286,501.716288,0.013865,501.730154,958.234009,297.325230,0.03287,0.065739,297.390969
3,8.26772,1.58,NaN,502.511144,99.21264,396.85056,28.937020,0.025929,502.537073,0.000000,502.537073,0.827286,501.709787,0.013865,501.723652,958.221591,297.317524,0.03287,0.098609,297.416133
4,8.26772,1.58,NaN,502.586328,99.21264,496.06320,37.204740,0.025933,502.612261,0.000000,502.612261,0.827286,501.784975,0.013869,501.798844,958.365197,297.406647,0.00000,0.131478,297.538125


### Incremental values

In [50]:
max_increment = int(np.ceil(rings_df.iloc[-1]["height_inch_total"]))

index_tuples = [(i, i+1) for i in range(-1, max_increment)]
index_tuples[0] = ("Below", 0)

index = pd.MultiIndex.from_tuples(index_tuples, names=["from", "to"])

In [51]:
increments_df = pd.DataFrame(index=index)

In [52]:
previous_top_boundary = 0
ring_column_list = []
for ring_idx in range(ring_count):
    ring_name = f"ring_{ring_idx}"
    ring_column_list.append(ring_name)
    increments_df[ring_name] = 0

    bottom_boundary = previous_top_boundary
    bottom_full_boundary = int(np.ceil(bottom_boundary))

    top_boundary = rings_df.loc[ring_idx]["height_inch_total"]
    top_full_boundary = int(np.floor(top_boundary))

    ring_incremental = rings_df.loc[ring_idx]["incremental"]

    for lower_boundary in range(bottom_full_boundary, top_full_boundary):
        increments_df.loc[(lower_boundary, lower_boundary+1), ring_name] = ring_incremental

    if bottom_boundary < bottom_full_boundary:
        
        diff = bottom_full_boundary - bottom_boundary;
        increments_df.loc[(bottom_full_boundary-1, bottom_full_boundary), ring_name] = ring_incremental * diff
    
    if top_boundary > top_full_boundary:
        diff = top_boundary - top_full_boundary;
        increments_df.loc[(top_full_boundary, top_full_boundary+1), ring_name] = ring_incremental * diff
    
    previous_top_boundary = top_boundary

increments_df["ring_volume_increments"] = increments_df[ring_column_list].sum(axis=1, skipna=True)

increments_df
    

,,ring_0,ring_1,ring_2,ring_3,ring_4,ring_volume_increments
from,to,,,,,,
Below,0,0.000000,0.0,0.0,0.0,0.000000,0.000000
0,1,297.380995,0.0,0.0,0.0,0.000000,297.380995
1,2,297.380995,0.0,0.0,0.0,0.000000,297.380995
2,3,297.380995,0.0,0.0,0.0,0.000000,297.380995
3,4,297.380995,0.0,0.0,0.0,0.000000,297.380995
...,...,...,...,...,...,...,...
492,493,0.000000,0.0,0.0,0.0,297.406647,297.406647
493,494,0.000000,0.0,0.0,0.0,297.406647,297.406647
494,495,0.000000,0.0,0.0,0.0,297.406647,297.406647


### Deadwood recapitulation

In [53]:
deadwood_column_list = []

#### Structural

In [54]:
for idx in range(len(structural_deadwood_add)):
    col_name = f"structural_deadwood_addition_{idx}"
    deadwood_column_list.append(col_name)
    increments_df[col_name] = 0

    current_deadwood = structural_deadwood_add[idx]

    if current_deadwood["from"] == "Below":
        increments_df.loc[("Below", 0), col_name] = current_deadwood["value"]
    else:
        bottom_boundary = current_deadwood["from"]
        bottom_full_boundary = int(np.ceil(bottom_boundary))

        top_boundary = current_deadwood["to"]
        top_full_boundary = int(np.floor(top_boundary))

        deadwood_incremental = current_deadwood["value"]

        for lower_boundary in range(bottom_full_boundary, top_full_boundary):
            increments_df.loc[(lower_boundary, lower_boundary+1), col_name] = deadwood_incremental

        if bottom_boundary < bottom_full_boundary:
            
            diff = bottom_full_boundary - bottom_boundary;
            increments_df.loc[(bottom_full_boundary-1, bottom_full_boundary), col_name] = deadwood_incremental * diff
        
        if top_boundary > top_full_boundary:
            diff = top_boundary - top_full_boundary;
            increments_df.loc[(top_full_boundary, top_full_boundary+1), col_name] = deadwood_incremental * diff

for idx in range(len(structural_deadwood_deduct)):
    col_name = f"structural_deadwood_deduction_{idx}"
    deadwood_column_list.append(col_name)
    increments_df[col_name] = 0

    current_deadwood = structural_deadwood_deduct[idx]

    if current_deadwood["from"] == "Below":
        increments_df.loc[("Below", 0), col_name] = -current_deadwood["value"]
    else:
        bottom_boundary = current_deadwood["from"]
        bottom_full_boundary = int(np.ceil(bottom_boundary))

        top_boundary = current_deadwood["to"]
        top_full_boundary = int(np.floor(top_boundary))

        deadwood_incremental = current_deadwood["value"]

        for lower_boundary in range(bottom_full_boundary, top_full_boundary):
            increments_df.loc[(lower_boundary, lower_boundary+1), col_name] = -deadwood_incremental

        if bottom_boundary < bottom_full_boundary:
            
            diff = bottom_full_boundary - bottom_boundary;
            increments_df.loc[(bottom_full_boundary-1, bottom_full_boundary), col_name] = -(deadwood_incremental * diff)
        
        if top_boundary > top_full_boundary:
            diff = top_boundary - top_full_boundary;
            increments_df.loc[(top_full_boundary, top_full_boundary+1), col_name] = -(deadwood_incremental * diff)

increments_df

,,ring_0,ring_1,ring_2,ring_3,ring_4,ring_volume_increments,structural_deadwood_addition_0,structural_deadwood_addition_1,structural_deadwood_addition_2,structural_deadwood_deduction_0,structural_deadwood_deduction_1
from,to,,,,,,,,,,,
Below,0,0.000000,0.0,0.0,0.0,0.000000,0.000000,2.0007,0.0000,0.0,0.0,0.000000
0,1,297.380995,0.0,0.0,0.0,0.000000,297.380995,0.0000,0.0486,0.0,0.0,0.000000
1,2,297.380995,0.0,0.0,0.0,0.000000,297.380995,0.0000,0.0486,0.0,0.0,-0.001155
2,3,297.380995,0.0,0.0,0.0,0.000000,297.380995,0.0000,0.0486,0.0,0.0,-0.001500
3,4,297.380995,0.0,0.0,0.0,0.000000,297.380995,0.0000,0.0486,0.0,0.0,-0.001500
...,...,...,...,...,...,...,...,...,...,...,...,...
492,493,0.000000,0.0,0.0,0.0,297.406647,297.406647,0.0000,0.0000,0.0,0.0,0.000000
493,494,0.000000,0.0,0.0,0.0,297.406647,297.406647,0.0000,0.0000,0.0,0.0,0.000000
494,495,0.000000,0.0,0.0,0.0,297.406647,297.406647,0.0000,0.0000,0.0,0.0,0.000000


#### Bottom

In [55]:
for idx in range(len(bottom_deadwood_add)):
    col_name = f"bottom_deadwood_addition_{idx}"
    deadwood_column_list.append(col_name)
    increments_df[col_name] = 0

    current_deadwood = bottom_deadwood_add[idx]

    if current_deadwood["from"] == "Below":
        increments_df.loc[("Below", 0), col_name] = current_deadwood["value"]
    else:
        bottom_boundary = current_deadwood["from"]
        bottom_full_boundary = int(np.ceil(bottom_boundary))

        top_boundary = current_deadwood["to"]
        top_full_boundary = int(np.floor(top_boundary))

        deadwood_incremental = current_deadwood["value"]

        for lower_boundary in range(bottom_full_boundary, top_full_boundary):
            increments_df.loc[(lower_boundary, lower_boundary+1), col_name] = deadwood_incremental

        if bottom_boundary < bottom_full_boundary:
            
            diff = bottom_full_boundary - bottom_boundary;
            increments_df.loc[(bottom_full_boundary-1, bottom_full_boundary), col_name] = deadwood_incremental * diff
        
        if top_boundary > top_full_boundary:
            diff = top_boundary - top_full_boundary;
            increments_df.loc[(top_full_boundary, top_full_boundary+1), col_name] = deadwood_incremental * diff

for idx in range(len(bottom_deadwood_deduct)):
    col_name = f"bottom_deadwood_deduction_{idx}"
    deadwood_column_list.append(col_name)
    increments_df[col_name] = 0

    current_deadwood = bottom_deadwood_deduct[idx]

    if current_deadwood["from"] == "Below":
        increments_df.loc[("Below", 0), col_name] = -current_deadwood["value"]
    else:
        bottom_boundary = current_deadwood["from"]
        bottom_full_boundary = int(np.ceil(bottom_boundary))

        top_boundary = current_deadwood["to"]
        top_full_boundary = int(np.floor(top_boundary))

        deadwood_incremental = current_deadwood["value"]

        for lower_boundary in range(bottom_full_boundary, top_full_boundary):
            increments_df.loc[(lower_boundary, lower_boundary+1), col_name] = -deadwood_incremental

        if bottom_boundary < bottom_full_boundary:
            
            diff = bottom_full_boundary - bottom_boundary;
            increments_df.loc[(bottom_full_boundary-1, bottom_full_boundary), col_name] = -(deadwood_incremental * diff)
        
        if top_boundary > top_full_boundary:
            diff = top_boundary - top_full_boundary;
            increments_df.loc[(top_full_boundary, top_full_boundary+1), col_name] = -(deadwood_incremental * diff)

increments_df

,,ring_0,ring_1,ring_2,ring_3,ring_4,ring_volume_increments,structural_deadwood_addition_0,structural_deadwood_addition_1,structural_deadwood_addition_2,structural_deadwood_deduction_0,structural_deadwood_deduction_1,bottom_deadwood_addition_0,bottom_deadwood_deduction_0,bottom_deadwood_deduction_1
from,to,,,,,,,,,,,,,,
Below,0,0.000000,0.0,0.0,0.0,0.000000,0.000000,2.0007,0.0000,0.0,0.0,0.000000,411.5441,0.0000,0.0000
0,1,297.380995,0.0,0.0,0.0,0.000000,297.380995,0.0000,0.0486,0.0,0.0,0.000000,0.0000,-17.1068,0.0000
1,2,297.380995,0.0,0.0,0.0,0.000000,297.380995,0.0000,0.0486,0.0,0.0,-0.001155,0.0000,0.0000,-6.4007
2,3,297.380995,0.0,0.0,0.0,0.000000,297.380995,0.0000,0.0486,0.0,0.0,-0.001500,0.0000,0.0000,0.0000
3,4,297.380995,0.0,0.0,0.0,0.000000,297.380995,0.0000,0.0486,0.0,0.0,-0.001500,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,493,0.000000,0.0,0.0,0.0,297.406647,297.406647,0.0000,0.0000,0.0,0.0,0.000000,0.0000,0.0000,0.0000
493,494,0.000000,0.0,0.0,0.0,297.406647,297.406647,0.0000,0.0000,0.0,0.0,0.000000,0.0000,0.0000,0.0000
494,495,0.000000,0.0,0.0,0.0,297.406647,297.406647,0.0000,0.0000,0.0,0.0,0.000000,0.0000,0.0000,0.0000


#### Accumulating deadwood deduction

In [56]:
increments_df["deadwood_deduction"] = increments_df[deadwood_column_list].sum(axis=1, skipna=True)

increments_df["net"] = increments_df["ring_volume_increments"] + increments_df["deadwood_deduction"]

increments_df

,,ring_0,ring_1,ring_2,ring_3,ring_4,ring_volume_increments,structural_deadwood_addition_0,structural_deadwood_addition_1,structural_deadwood_addition_2,structural_deadwood_deduction_0,structural_deadwood_deduction_1,bottom_deadwood_addition_0,bottom_deadwood_deduction_0,bottom_deadwood_deduction_1,deadwood_deduction,net
from,to,,,,,,,,,,,,,,,,
Below,0,0.000000,0.0,0.0,0.0,0.000000,0.000000,2.0007,0.0000,0.0,0.0,0.000000,411.5441,0.0000,0.0000,413.544800,413.544800
0,1,297.380995,0.0,0.0,0.0,0.000000,297.380995,0.0000,0.0486,0.0,0.0,0.000000,0.0000,-17.1068,0.0000,-17.058200,280.322795
1,2,297.380995,0.0,0.0,0.0,0.000000,297.380995,0.0000,0.0486,0.0,0.0,-0.001155,0.0000,0.0000,-6.4007,-6.353255,291.027740
2,3,297.380995,0.0,0.0,0.0,0.000000,297.380995,0.0000,0.0486,0.0,0.0,-0.001500,0.0000,0.0000,0.0000,0.047100,297.428095
3,4,297.380995,0.0,0.0,0.0,0.000000,297.380995,0.0000,0.0486,0.0,0.0,-0.001500,0.0000,0.0000,0.0000,0.047100,297.428095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,493,0.000000,0.0,0.0,0.0,297.406647,297.406647,0.0000,0.0000,0.0,0.0,0.000000,0.0000,0.0000,0.0000,0.000000,297.406647
493,494,0.000000,0.0,0.0,0.0,297.406647,297.406647,0.0000,0.0000,0.0,0.0,0.000000,0.0000,0.0000,0.0000,0.000000,297.406647
494,495,0.000000,0.0,0.0,0.0,297.406647,297.406647,0.0000,0.0000,0.0,0.0,0.000000,0.0000,0.0000,0.0000,0.000000,297.406647


### Volume accumulation

In [57]:
increments_df["accumulated_barrels"] = 0

for i in range(len(index_tuples)):
    curr_idx = index_tuples[i]
    acc = increments_df.loc[curr_idx]["net"]
    if i > 0:
        prev_idx = index_tuples[i-1]
        acc = acc + increments_df.loc[prev_idx]["accumulated_barrels"]
    increments_df.loc[curr_idx, "accumulated_barrels"] = acc

increments_df


,,ring_0,ring_1,ring_2,ring_3,ring_4,ring_volume_increments,structural_deadwood_addition_0,structural_deadwood_addition_1,structural_deadwood_addition_2,structural_deadwood_deduction_0,structural_deadwood_deduction_1,bottom_deadwood_addition_0,bottom_deadwood_deduction_0,bottom_deadwood_deduction_1,deadwood_deduction,net,accumulated_barrels
from,to,,,,,,,,,,,,,,,,,
Below,0,0.000000,0.0,0.0,0.0,0.000000,0.000000,2.0007,0.0000,0.0,0.0,0.000000,411.5441,0.0000,0.0000,413.544800,413.544800,413.544800
0,1,297.380995,0.0,0.0,0.0,0.000000,297.380995,0.0000,0.0486,0.0,0.0,0.000000,0.0000,-17.1068,0.0000,-17.058200,280.322795,693.867595
1,2,297.380995,0.0,0.0,0.0,0.000000,297.380995,0.0000,0.0486,0.0,0.0,-0.001155,0.0000,0.0000,-6.4007,-6.353255,291.027740,984.895334
2,3,297.380995,0.0,0.0,0.0,0.000000,297.380995,0.0000,0.0486,0.0,0.0,-0.001500,0.0000,0.0000,0.0000,0.047100,297.428095,1282.323429
3,4,297.380995,0.0,0.0,0.0,0.000000,297.380995,0.0000,0.0486,0.0,0.0,-0.001500,0.0000,0.0000,0.0000,0.047100,297.428095,1579.751524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,493,0.000000,0.0,0.0,0.0,297.406647,297.406647,0.0000,0.0000,0.0,0.0,0.000000,0.0000,0.0000,0.0000,0.000000,297.406647,146978.649669
493,494,0.000000,0.0,0.0,0.0,297.406647,297.406647,0.0000,0.0000,0.0,0.0,0.000000,0.0000,0.0000,0.0000,0.000000,297.406647,147276.056316
494,495,0.000000,0.0,0.0,0.0,297.406647,297.406647,0.0000,0.0000,0.0,0.0,0.000000,0.0000,0.0000,0.0000,0.000000,297.406647,147573.462963


## Output tables

### Run sheet

All of the previous data is then compile into a runsheet, so a table with information on incremental values (BBL/inch), accumulated values and shortened, to omit repeating rows.

In [58]:
run_sheet_list = []

target_colls = ["ring_volume_increments", "deadwood_deduction", "net"]
target_acc_col = "accumulated_barrels"

rs_bottom_idx = 0
increment_count = len(increments_df)

while rs_bottom_idx < increment_count:
    initial_row = increments_df.iloc[rs_bottom_idx][target_colls]
    initial_row_acc = increments_df.iloc[rs_bottom_idx][target_acc_col]

    no_of_inc = 1

    candidate = {
        "From": initial_row.name[0], 
        "To": initial_row.name[1], 
        "No. of increments": 1, 
        "Vol. + Corr. incremental": initial_row["ring_volume_increments"],
        "Deadwood": initial_row["deadwood_deduction"],
        "Net": initial_row["net"],
        "Accumulated barrels": initial_row_acc}

    rs_explore_idx = rs_bottom_idx + 1

    while rs_explore_idx < increment_count:
        explore_row = increments_df.iloc[rs_explore_idx][target_colls]
        explore_row_acc = increments_df.iloc[rs_explore_idx][target_acc_col]

        if not initial_row.equals(explore_row):
            break
        else:
            candidate["To"] = explore_row.name[1]
            no_of_inc = no_of_inc + 1
            candidate["Accumulated barrels"] = explore_row_acc
            rs_explore_idx = rs_explore_idx + 1

    candidate["No. of increments"] = no_of_inc
    run_sheet_list.append(candidate)
    rs_bottom_idx = rs_explore_idx

run_sheet_df = pd.DataFrame.from_records(run_sheet_list, index=["From", "To"])

run_sheet_df

,,No. of increments,Vol. + Corr. incremental,Deadwood,Net,Accumulated barrels
From,To,,,,,
Below,0,1,0.000000,413.544800,413.544800,413.544800
0,1,1,297.380995,-17.058200,280.322795,693.867595
1,2,1,297.380995,-6.353255,291.027740,984.895334
2,17,15,297.380995,0.047100,297.428095,5446.316755
17,18,1,297.380995,0.046760,297.427755,5743.744510
18,26,8,297.380995,0.046600,297.427595,8123.165268
26,27,1,297.380995,0.046615,297.427610,8420.592878
27,41,14,297.380995,0.047100,297.428095,12584.586204
41,42,1,297.380995,0.060344,297.441339,12882.027542


### Calibration table

Finaly, the last output table gives information about barrels per inch.

In [59]:
calibration_table_list = []

for idx in range(increment_count):
    row = increments_df.iloc[idx]
    
    to = row.name[1]

    calibration_table_list.append({"FT": int(np.floor(to / 12)), "IN": int(to % 12), "Barrel": int(row["accumulated_barrels"])})

calibration_table_df = pd.DataFrame.from_records(calibration_table_list, index=["FT", "IN"])
calibration_table_df

Barrel
FT IN        
0  0      413
   1      693
   2      984
   3     1282
   4     1579
...       ...
41 1   146978
   2   147276
   3   147573
   4   147870
   5   147889

[498 rows x 1 columns]

## Report generation

The previously calculated tables are then exported to an excel file.

In [60]:
excel_writer = pd.ExcelWriter("report.xlsx")

run_sheet_df.to_excel(excel_writer, sheet_name="Run Sheet")

rows_in_page = 12 * 5;

page_count = int(np.ceil(increment_count/rows_in_page))

for page in range(page_count):
    bottom_limit = rows_in_page * page
    top_limit = min(bottom_limit + rows_in_page, increment_count)
    col = page * 3

    calibration_table_df.iloc[bottom_limit:top_limit].to_excel(excel_writer, sheet_name="Calibration Table", startcol=col)


excel_writer.save()

## Serialization

### Run sheet

In [61]:
run_sheet = []

for row_idx in range(len(run_sheet_df)):
    row = run_sheet_df.iloc[row_idx]
    run_sheet.append({
        "from": row.name[0],
        "to": row.name[1],
        "numberOfIncrements": row["No. of increments"],
        "incrementalVolumeCorrected": row["Vol. + Corr. incremental"],
        "deadwood": row["Deadwood"],
        "net": row["Net"],
        "accumulatedBarrels": row["Accumulated barrels"]
    })


In [62]:
calibration_table = []

for row_idx in range(len(calibration_table_df)):
    row = calibration_table_df.iloc[row_idx]
    calibration_table.append({
        "ft": row.name[0],
        "in": row.name[1],
        "accumulatedBarrels": row["Barrel"]
    })